In [7]:
import torch
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingClassifier

data, y = torch.load('data/data_v2.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

score_model = {}
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )
np.random.seed(0)

std = StandardScaler()
bagging_svm = BaggingClassifier(base_estimator=SVC(probability=True))
bagging_svm.set_params(
    **{'n_estimators': 99, 'base_estimator__C': 36.06241178040971, 'base_estimator__gamma': 0.004735786412937079}
)
lr = LogisticRegression()
adaboost = AdaBoostClassifier(base_estimator=SVC(probability=True))
adaboost_tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
adaboost_tree.set_params(**{'n_estimators': 349, 'learning_rate': 0.01, 'base_estimator__max_depth': 36,
                            'base_estimator__min_samples_split': 189, 'base_estimator__min_samples_leaf': 1,
                            'base_estimator__max_features': 25, 'base_estimator__max_leaf_nodes': 471}
                         )
adaboost.set_params(
    **{'n_estimators': 65, 'learning_rate': 0.03694467169486954, 'base_estimator__C': 0.28335389417624707,
       'base_estimator__gamma': 0.007507640146291138}
)
gdbt = GradientBoostingClassifier()
gdbt.set_params(
    **{'n_estimators': 300, 'learning_rate': 0.06504234821394586, 'max_depth': 3, 'min_samples_split': 2,
       'min_samples_leaf': 1, 'max_features': 14, 'max_leaf_nodes': 891}

)
knn = KNeighborsClassifier()
knn.set_params(**{'n_neighbors': 6, 'p': 2, 'weights': 'uniform'}
               )
rf = RandomForestClassifier()
rf.set_params(
    **{'n_estimators': 276, 'max_depth': 99, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 21,
       'max_leaf_nodes': 31}
)

svc = SVC(probability=True)
svc.set_params(**{'C': 1.0121047403763643, 'gamma': 0.17953963922087096}
               )
tree = DecisionTreeClassifier()
tree.set_params(
    **{'max_depth': 3, 'min_samples_split': 103, 'min_samples_leaf': 1, 'max_features': 24}

)
model_list = [('lr', lr), ('ada', adaboost), ('gdbt', gdbt),
              ('knn', knn), ('svc', svc), ('tree', tree),
              ('bagging_svm', bagging_svm), ]

from itertools import combinations

for i in range(2, len(model_list) + 1):
    for models in combinations(model_list, i):
        voting_model = VotingClassifier(list(models), voting='soft', n_jobs=-1,
                                        verbose=2)
        pipe = Pipeline([
            ('std', std),
            ('voting', voting_model)
        ])

        score = cross_val_score(pipe, train_data, y, n_jobs=-1).mean()
        print(models,score)
        pipe.fit(train_data, y)
        score_model[-score] = pipe

import pandas as pd
import heapq

keys = list(score_model.keys())
heapq.heapify(keys)
same_score = set()
for i in range(len(keys)):
    _ = heapq.heappop(keys)
    if _ in same_score:
        continue
    else:
        same_score.add(_)
    y_test = score_model[_].predict(test_data.values)
    result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
    result.to_csv(f'rank{i}.csv')

(('lr', LogisticRegression()), ('ada', AdaBoostClassifier(base_estimator=SVC(C=0.28335389417624707,
                                      gamma=0.007507640146291138,
                                      probability=True),
                   learning_rate=0.03694467169486954, n_estimators=65))) 0.824913690289373
(('lr', LogisticRegression()), ('gdbt', GradientBoostingClassifier(learning_rate=0.06504234821394586, max_features=14,
                           max_leaf_nodes=891, n_estimators=300))) 0.8417425145941875
(('lr', LogisticRegression()), ('knn', KNeighborsClassifier(n_neighbors=6))) 0.8248948590797814
(('lr', LogisticRegression()), ('svc', SVC(C=1.0121047403763643, gamma=0.17953963922087096, probability=True))) 0.8215554579122465
(('lr', LogisticRegression()), ('tree', DecisionTreeClassifier(max_depth=3, max_features=24, min_samples_split=103))) 0.8204193082669009
(('lr', LogisticRegression()), ('bagging_svm', BaggingClassifier(base_estimator=SVC(C=36.06241178040971,
            

KeyboardInterrupt: 